In [1]:
# BadukPop Analytics Dashboard

# setup
import time, os, calendar, sys, datetime
import envkey
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1640217600
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_retention_sql(game,start_time,end_time,time_now):
    return """SELECT  1337274000 + 14*86400*(1+FLOOR((account_creation_time + 8*86400 - 1337274000)/(14*86400))) AS pvp_week_end,
    IF(country_tier IN ('1','2'), 'T12', 'T34') AS tier,
    IF(IFNULL(acquisition_campaign,'MISSING') LIKE '%s' OR acquisition_campaign LIKE '%s', 'Paid', 'Free') AS acquisition_type,
    COUNT(1) as N,
    ROUND(SUM(IF(toc_level >= 2,1,0)) / SUM(1), 2) AS `TOC L2`,
    ROUND(SUM(IF(`returned_24-48h`,1,0)) / SUM(1), 2) AS `1-day Return`,
    ROUND(SUM(IF(`returned_48-72h`,1,0)) / SUM(1), 2) AS `2-day Return`,
    ROUND(SUM(IF(`returned_72-96h`,1,0)) / SUM(1), 2) AS `3-day Return`,
    ROUND(SUM(IF(`returned_120-144h`,1,0)) / SUM(1), 2) AS `5-day Return`,
    ROUND(SUM(IF(`returned_168-192h`,1,0)) / SUM(1), 2) AS `7-day Return`
    FROM `%s_upcache`
    WHERE account_creation_time + 8*86400 >= %d - 14*86400
    AND account_creation_time + 8*86400 < LEAST(%d, %d)
    GROUP BY `pvp_week_end`, tier, acquisition_type
    HAVING pvp_week_end < %d;""" % ('%%_SRD','%%_GG',game,start_time,time_now,end_time,time_now)

In [2]:
time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

retention_data = {'tr':{},'dv':{}}
retention_weeks = []
for game in ('tr','dv'):
    game_cache = '%s_upcache' % game
    engine = create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{game_cache}', connect_args={'connect_timeout': 10})
    retention_sql = get_retention_sql(game,start_time,end_time,time_now)
    with engine.connect() as con:
        rs = con.execute(retention_sql)
        for row in rs:
            this_data = {}
            this_data['week'] = int(row[0])
            if int(row[0]) not in retention_weeks:
                retention_weeks.append(int(row[0]))
            this_data['tier'] = row[1]
            this_data['type'] = row[2]
            this_data['n'] = str(row[3])
            this_data['tocL2'] = '{0:.0%}'.format(row[4])
            this_data['1D'] = '{0:.0%}'.format(row[5])
            this_data['2D'] = '{0:.0%}'.format(row[6])
            this_data['3D'] = '{0:.0%}'.format(row[7])
            this_data['5D'] = '{0:.0%}'.format(row[8])
            this_data['7D'] = '{0:.0%}'.format(row[9])
            data_index = str(int(row[0])) + row[1]
            retention_data[game][data_index] = this_data

[1639674000, 1640883600, 1642093200]


In [3]:
retention_weeks.sort()
#print(retention_weeks)
#print(retention_data)
for game in ('tr','dv'):
    print('%s Retention (for accounts 8+ days old) * only updated every 2 weeks' % game.upper())
    header = "{0:12} {1:5} {2:6} {3:6} {4:8} {5:8} {6:8} {7:8} {8:8} {9:8}".format('Week', 'Tier', 'Type', 'N', 'TOC L2', '1-day', '2-day','3-day','5-day','7-day')
    print(header)
    for week in retention_weeks:
        for tier in ('T12','T34'):
            key = str(week) + tier
            this_data = retention_data[game][key]
            formatted_week = formatted_date = datetime.datetime.utcfromtimestamp(week).strftime('%Y-%m-%d')
            this_row = "{0:12} {1:5} {2:6} {3:6} {4:8} {5:8} {6:8} {7:8} {8:8} {9:8}".format(formatted_week, this_data['tier'],this_data['type'],this_data['n'],this_data['tocL2'],
                                                                                           this_data['1D'],this_data['2D'],this_data['3D'],this_data['5D'],this_data['7D'])
            print(this_row)
    print('')

TR Retention (for accounts 8+ days old) * only updated every 2 weeks
Week         Tier  Type   N      TOC L2   1-day    2-day    3-day    5-day    7-day   
2021-12-16   T12   Free   11     27%      18%      9%       9%       9%       0%      
2021-12-16   T34   Free   34     59%      18%      21%      18%      18%      18%     
2021-12-30   T12   Free   33     45%      12%      9%       15%      12%      12%     
2021-12-30   T34   Free   61     69%      28%      28%      25%      20%      18%     
2022-01-13   T12   Free   18     56%      33%      28%      22%      22%      11%     
2022-01-13   T34   Free   68     71%      22%      18%      12%      13%      12%     

DV Retention (for accounts 8+ days old) * only updated every 2 weeks
Week         Tier  Type   N      TOC L2   1-day    2-day    3-day    5-day    7-day   
2021-12-16   T12   Free   1      100%     0%       0%       0%       0%       0%      
2021-12-16   T34   Free   5      80%      20%      0%       20%      20%      